In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ase.units import Bohr
from pathlib import Path

In [ ]:
results_dimer_folder = Path(
    "/home/moritz/SCME/scme_reference_calculations/compare_against_fortran_code/results/dimer"
)
scme_truth = pd.read_json(results_dimer_folder / "scme_truth.json")

results = pd.read_json(results_dimer_folder / "results.json")

results_dimer_old_folder = Path(
    "/home/moritz/SCME/scme_reference_calculations/compare_against_fortran_code/results/dimer_old"
)
results_old = pd.read_json(results_dimer_old_folder / "results.json")

plot_dir = Path("./plots/dimer")
plot_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
dipoles_dimer = []
quadrupoles_dimer = []
samples = []

for p in results_dimer_folder.glob("[0-9]*"):
    sample = int(p.name)
    d = np.load(p / "dipoles.npy")
    q = np.load(p / "quadrupoles.npy")
    dipoles_dimer.append(d)
    quadrupoles_dimer.append(q)
    samples.append(sample)

dipoles_dimer = np.array(dipoles_dimer)
quadrupoles_dimer = np.array(quadrupoles_dimer)

dipoles_dimer = dipoles_dimer[np.argsort(samples)]
quadrupoles_dimer = quadrupoles_dimer[np.argsort(samples)]

# Plot the total potential energy

In [ ]:
ax = plt.gca()
ax.plot(scme_truth["sample"], scme_truth["energy"], marker="o", ms=9, label="fortran")
ax.plot(
    results_old["sample"],
    results_old["energy_tot"],
    ls="None",
    marker="s",
    label="old_cpp",
)
std = np.std(results["energy_tot"] - scme_truth["energy"])
print(f"{std = }")

ax.set_title("Dimer energy.")
ax.plot(results["sample"], results["energy_tot"], ls="None", marker=".", label="moritz")
ax.set_xlabel("sample")
ax.set_ylabel("energy [eV]")
ax.legend()
plt.tight_layout()
plt.savefig(plot_dir / "energy.png", dpi=300)
plt.show()
plt.close()

# Plot the components of the total dipole moment

In [ ]:
for comp in [0, 1, 2]:
    ax = plt.gca()
    ax.set_title(f"Total dipole moment component {comp+1}")

    d_truth = np.array([d[0][comp] + d[1][comp] for d in scme_truth["dipole"]])
    d_res = [d[comp] for d in results["dipole"]]
    d_res_old = [d[comp] / Bohr for d in results_old["dipole"]]

    std = np.std(d_truth - d_res)
    print(f"{std = }")

    ax.plot(
        scme_truth["sample"],
        d_truth,
        marker="o",
        ms=9,
        label="fortran",
    )
    ax.plot(
        results["sample"],
        d_res,
        marker="s",
        label="old_cpp",
    )
    ax.plot(
        results_old["sample"],
        d_res_old,
        marker=".",
        label="moritz",
    )
    ax.set_ylabel(r"$\mu_{" + str(comp + 1) + "}~[e a_0]$")
    ax.set_xlabel("sample")
    plt.legend()
    plt.tight_layout()
    plt.savefig(plot_dir / f"dms_component_{comp}.png", dpi=300)
    plt.show()
    plt.close()

# Plot the components of the total quadrupole moment

In [ ]:
for comp1 in [0, 1, 2]:
    for comp2 in [0, 1, 2]:
        ax = plt.gca()
        ax.set_title(f"Total quadrupole moment component {comp1+1} {comp2+1}")

        q_truth = np.array(
            [q[0][comp1][comp2] + q[1][comp1][comp2] for q in scme_truth["quadrupole"]]
        )

        q_res_old = np.array(
            [q[comp1][comp2] / Bohr**2 for q in results_old["quadrupole"]]
        )
        q_res = np.array([q[comp1][comp2] for q in results["quadrupole"]])

        std = np.std(q_truth - q_res)
        print(f"{std = }")

        ax.plot(
            scme_truth["sample"],
            q_truth,
            marker="o",
            ms=9,
            label="fortran",
        )
        ax.plot(
            results_old["sample"],
            q_res_old,
            marker="s",
            label="old_cpp",
        )
        ax.plot(
            results["sample"],
            q_res,
            marker=".",
            label="moritz",
        )
        ax.set_ylabel(r"$\theta_{" + str(comp1 + 1) + str(comp2 + 1) + "}~[e a_0^2]$")
        ax.set_xlabel("sample")
        plt.legend()
        plt.tight_layout()
        plt.savefig(plot_dir / f"qms_component_{comp1}_{comp2}.png", dpi=300)
        plt.show()
        plt.close()

# Plot the individual dipole moments

In [ ]:
for i_atom in range(2):
    for comp in [0, 1, 2]:
        ax = plt.gca()
        ax.set_title(f"Dipole moment atom {i_atom+1}, component {comp+1}")

        d_truth = [d[i_atom][comp] for d in scme_truth["dipole"]]
        d_res = dipoles_dimer[:, i_atom, comp]

        ax.plot(
            scme_truth["sample"], d_truth, marker="o", ms=9, label="fortran", color="C0"
        )

        ax.plot(results["sample"], d_res, marker=".", label="moritz", color="C2")

        ax.set_ylabel(r"$\mu_{" + str(comp + 1) + "}~[e a_0]$")
        ax.set_xlabel("sample")
        plt.legend()
        plt.tight_layout()
        plt.savefig(plot_dir / f"dms_component_atom_{i_atom}_{comp}.png", dpi=300)
        plt.show()
        plt.close()

# Plot individual quadrupole moments

In [ ]:
for i_atom in range(2):
    for comp1 in [0, 1, 2]:
        for comp2 in [0, 1, 2]:
            ax = plt.gca()
            ax.set_title(
                f"Quadrupole moment atom {i_atom+1}, component {comp1+1} {comp2+1}"
            )

            q_truth = np.array(
                [q[i_atom][comp1][comp2] for q in scme_truth["quadrupole"]]
            )
            q_res = quadrupoles_dimer[:, i_atom, comp1, comp2]

            ax.plot(
                scme_truth["sample"],
                q_truth,
                marker="o",
                ms=9,
                label="fortran",
                color="C0",
            )

            ax.plot(results["sample"], q_res, marker=".", label="moritz", color="C2")

            ax.set_ylabel(r"$\mu_{" + str(comp + 1) + "}~[e a_0]$")
            ax.set_xlabel("sample")
            plt.legend()
            plt.tight_layout()
            plt.savefig(plot_dir / f"qms_component_atom_{i_atom}_{comp}.png", dpi=300)
            plt.show()
            plt.close()